# Phase 3 - Function Calling

## About this phase
In this challenge you have to make sure, that your API is able to answer questions, that neither the model knows, nor are the answers hidden in your custom data. But you can make your model aware of a bunch of APIs that can be used to call the APIs. 
So go, make the model aware of the APIs and build an application that call the APIs whenever required!


If not already done run this in the top level folder:
```
pip install -r requirements.txt
```




In [2]:
import json
import os
from enum import Enum
from pydantic import BaseModel
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

Found Azure OpenAI API Base Endpoint: https://cog-mdhi5lxwmoqli.openai.azure.com/


This is the object model for receiving questions.

In [3]:

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_or_false = "true_or_false"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


## Sample Function 

In [11]:
import requests

smoorghApi = "https://smoorgh-api.happypebble-f6fb3666.northeurope.azurecontainerapps.io/"

def get_movie_rating(title):
    try:
        headers = {"title": title}
        response = requests.get(f"{smoorghApi}rating", headers=headers)
        print('The api response for rating is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find a rating for that movie."

def get_movie_year(title):
    try:
        headers = {"title": title}
        response = requests.get(f"{smoorghApi}year", headers=headers)
        print('The api response for year is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find a year for that movie."
    
def get_movie_actor(title):
    try:
        headers = {"title": title}
        response = requests.get(f"{smoorghApi}actor", headers=headers)
        print('The api response for actor is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find an actor for that movie."
    
def get_movie_location(title):
    try:
        headers = {"title": title}
        response = requests.get(f"{smoorghApi}location", headers=headers)
        print('The api response for location is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find a location for that movie."

def get_movie_genre(title):
    try:
        headers = {"title": title}
        response = requests.get(f"{smoorghApi}genre", headers=headers)
        print('The api response for genre is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find a genre for that movie."

title_param = "Hobbiton: The Heist of the Silver Dragon"

print(get_movie_rating(title_param))

print(get_movie_year(title_param))

print(get_movie_actor(title_param))

print(get_movie_location(title_param))

print(get_movie_genre(title_param))

The api response for rating is: 8.5
8.5
The api response for year is: 2025
2025
The api response for actor is: Orin Stone
Orin Stone
The api response for location is: Glimmerholme
Glimmerholme
The api response for genre is: Fantasy
Fantasy


Make the model aware of the function


In [12]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_movie_rating",
                "description": "Gets the rating of a movie",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The movie name. The movie name should be a string without quotation marks.",
                        }
                    },
                    "required": ["title"],
                },
            }
        },
         {
            "type": "function",
            "function": {
                "name": "get_movie_location",
                "description": "Gets the location of a movie",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The movie name. The movie name should be a string without quotation marks.",
                        }
                    },
                    "required": ["title"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_movie_genre",
                "description": "Gets the genre of a movie",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The movie name. The movie name should be a string without quotation marks.",
                        }
                    },
                    "required": ["title"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_movie_actor",
                "description": "Gets the actor of a movie",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The movie name. The movie name should be a string without quotation marks.",
                        }
                    },
                    "required": ["title"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_movie_year",
                "description": "Gets the year of a movie",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The movie name. The movie name should be a string without quotation marks.",
                        }
                    },
                    "required": ["title"],
                },
            }
        }
    ]   
available_functions = {
            "get_movie_rating": get_movie_rating,
            "get_movie_location": get_movie_location,
            "get_movie_genre": get_movie_genre,
            "get_movie_actor": get_movie_actor,
            "get_movie_year": get_movie_year
        } 

# YOUR Mission: 
Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service. 
Ensure the answers provided are correct and in the correct format.


In [21]:

async def ask_question(ask: Ask):
    """
    Ask a question
    """
    
    question = ask.question
    question_type = ask.type

    messages= [{"role" : "assistant", "content" : question}
        , { "role" : "system", "content" : "Answer the question of the user and use the tools available to you. Only return the answer without encapsulating it in a sentence to the question what the tool returned in the content field, no bs."}
    ]
    first_response = client.chat.completions.create(
        model = deployment_name,
        messages = messages,
        tools = functions,
        tool_choice = "auto",
    )

    print(first_response)
    response_message = first_response.choices[0].message
    tool_calls = response_message.tool_calls

     # Step 2: check if GPT wanted to call a function
    if tool_calls:
        print("Recommended Function call:")
        print(tool_calls)
        print()
    
        # Step 3: call the function
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            # verify function exists
            if function_name not in available_functions:
                return "Function " + function_name + " does not exist"
            else:
                print("Calling function: " + function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(function_args)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
            print("Addding this message to the next prompt:") 
            print (messages)
            
             # extend conversation with function response
            second_response = client.chat.completions.create(
                model = deployment_name,
                temperature=0,
                messages = messages)  # get a new response from the model where it can see the function response
            
            print("second_response")
            
            return second_response.choices[0].message.content
            

   

Use this snippet to try your method with several questions.

In [23]:

ask = Ask(question="Which country does 'Hobbiton: The Heist of the Silver Dragon' take place in?", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer )

ask = Ask(question="In the Marvel Cinematic Universe, who is the actor that brings Tony Stark to life? Robert Downey Jr., Chris Hemsworth, Chris Evans, Mark Ruffalo", type=QuestionType.multiple_choice)
answer = await ask_question(ask)
print('Answer:', answer )

ChatCompletion(id='chatcmpl-A8Rg0PnmlJRdZpiOmB8lH6kBj0tPA', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rdtHKcV82Qb3GzEVp40Oejlb', function=Function(arguments='{"title":"Hobbiton: The Heist of the Silver Dragon"}', name='get_movie_location'), type='function')]), content_filter_results={})], created=1726576376, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=26, prompt_tokens=279, total_tokens=305), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {}}])
Recommended Function call:
[ChatCompletionMessageToolCall(id='call_rdtHKcV82Qb3GzEVp40Oejlb', function=Function(arguments='{"title":"Hobbiton: The Heist of the Silver Dragon"}', name='get_movie_location'), type='function')]

Calling function: get_movie

Make sure you transfer your code changes into main.py (or additional files). Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase3 $AZURE_ENV_NAME
```
Make sure to provide the URL of your endpoint in the team portal!